# Würfel Orientierung ermitteln

In [69]:
import numpy as np
import cv2 as cv
from PIL import Image
import matplotlib.pyplot as plt

## Definition des KSYS
- x: blau
- y: rot
- z: weiß

## Testszenario
Es wird ein Testszenario definiert:
1. Der Würfel liegt mit der gelben Seite nach oben auf dem Tisch
2. Der Würfel wird gegriffen und in die Kamera gehalten, sodass die weiße Seite sichtbar ist
3. Der Würfel wird um 90° gedreht, sodass die blaue Seite sichtbar ist
4. Der Würfel wird um 180° gedreht, sodass die gegenüberliegende grüne Seite sichtbar ist

Theoretisch reicht bereits die erste 90° Drehung, um das KSys zu bestimmen.

### Szenario 1

In [70]:
image1 = {
    "parentcolor": "W",
    "matrix": "OBORWORWR"
}

# Erste rotation, um das globale KSYS
rotation1 = {
    "axis": "x",
    "steps": 1,
}

image2 = {
    "parentcolor": "B",
    "matrix": "YWWGBYBYW"
}

# Zweite rotation, um das globale KSYS
rotation2 = {
    "axis": "y",
    "steps": 2,
}

image3 = {
    "parentcolor": "G",
    "matrix": "BRRBGGYGG"
}

### Szenario 2

In [71]:
image1_2 = {
    "parentcolor": "O",
    "matrix": "GWBGOYROW"
}

# Erste rotation, um das globale KSYS
rotation1_2 = {
    "axis": "x",
    "steps": 1,
}

image2_2 = {
    "parentcolor": "G",
    "matrix": "YBBGGRGGR"
}

# Zweite rotation, um das globale KSYS
rotation2_2 = {
    "axis": "y",
    "steps": 2,
}

image3_2 = {
    "parentcolor": "B",
    "matrix": "WYWWBYYGB"
}

### Lösung
gilt für beide Szenarien

In [72]:
solution = "OORBWWORRGWBGOYROWYBBGGRGGRWOYWRRYBBBGYYBWWYWORGYYOGBO"

## Version 1
Aktuelle Orientierung des Würfels ermitteln

In [73]:
def get_axis_by_facecolor(image: dict):
    """Ordnet der Seite eine Achse zu"""
    if image["parentcolor"] == "W":
        return "z"
    elif image["parentcolor"] == "Y":
        return "-z"
    elif image["parentcolor"] == "R":
        return "x"
    elif image["parentcolor"] == "O":
        return "-x"
    elif image["parentcolor"] == "B":
        return "y"
    elif image["parentcolor"] == "G":
        return "-y"
    else:
        raise ValueError("Fehler in 'detect_axis_by_facecolor': Es wurde keine gültige Parentcolor erkannt")
    
def get_cube_orientation(image1: dict, image2: dict, rotation:dict):
    """Ermittelt die Gesamtorientierung aus den ersten zwei Bildern.
    ACHTUNG: Es wird immer angenommen, dass die erste Rotation um die globale x-Achse gemacht wurde"""

    if image1["axis"] == "z": # weiß links
        if image2["axis"] == "x": # rot oben
            x = [0,0,1] # x zeigt in globale z-Richtung
            y = [-1,0,0] # y zeigt in globale -x-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "-x": # orange oben
            x = [0,0,1] # x zeigt in globale -z-Richtung
            y = [1,0,0] # y zeigt in globale x-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "y": # blau oben
            x = [1,0,0] # x zeigt in globale x-Richtung
            y = [0,0,1] # y zeigt in globale z-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "-y": # grün oben
            x = [-1,0,0] # x zeigt in globale -x-Richtung
            y = [0,0,-1] # y zeigt in globale -z-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung

    elif image1["axis"] == "-z": # gelb links
        if image2["axis"] == "x": # rot oben
            x = [0,0,1] # x zeigt in globale z-Richtung
            y = [1,0,0] # y zeigt in globale x-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung
        elif image2["axis"] == "-x": # orange oben
            x = [0,0,-1] # x zeigt in globale -z-Richtung
            y = [-1,0,0] # y zeigt in globale -x-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung
        elif image2["axis"] == "y": # blau oben
            x = [-1,0,0] # x zeigt in globale -x-Richtung
            y = [0,0,1] # y zeigt in globale z-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung
        elif image2["axis"] == "-y": # grün oben
            x = [1,0,0] # x zeigt in globale x-Richtung
            y = [0,0,-1] # y zeigt in globale -z-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung

    elif image1["axis"] == "x": # rot links
        if image2["axis"] == "z": # weiß oben
            x = [0,-1,0] # x zeigt in globale -y-Richtung
            y = [1,0,0] # y zeigt in globale x-Richtung
            z = [0,0,1] # z zeigt in globale z-Richtung
        elif image2["axis"] == "-z": # gelb oben
            x = [0,-1,0] # x zeigt in globale -y-Richtung
            y = [-1,0,0] # y zeigt in globale -x-Richtung
            z = [0,0,-1] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "y": # blau oben
            x = [0,-1,0] # x zeigt in globale -y-Richtung
            y = [0,0,1] # y zeigt in globale z-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
        elif image2["axis"] == "-y": # grün oben
            x = [0,-1,0] # x zeigt in globale -y-Richtung
            y = [0,0,-1] # y zeigt in globale -z-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung

    elif image1["axis"] == "-x": # orange links
        if image2["axis"] == "z": # weiß oben
            x = [0,1,0] # x zeigt in globale y-Richtung
            y = [-1,0,0] # y zeigt in globale -x-Richtung
            z = [0,0,1] # z zeigt in globale z-Richtung
        elif image2["axis"] == "-z": # gelb oben
            x = [0,1,0] # x zeigt in globale y-Richtung
            y = [1,0,0] # y zeigt in globale x-Richtung
            z = [0,0,-1] # z zeigt in globale -z-Richtung
        elif image2["axis"] == "y": # blau oben
            x = [0,1,0] # x zeigt in globale y-Richtung
            y = [0,0,1] # y zeigt in globale z-Richtung
            z = [0,1,0] # z zeigt in globale y-Richtung
        elif image2["axis"] == "-y": # grün oben
            x = [0,1,0] # x zeigt in globale y-Richtung
            y = [0,0,-1] # y zeigt in globale -z-Richtung
            z = [0,-1,0] # z zeigt in globale -y-Richtung
    
    elif image1["axis"] == "y": # blau links
        if image2["axis"] == "z": # weiß oben
            x = [-1,0,0] # x zeigt in globale x-Richtung
            y = [0,-1,0] # y zeigt in globale -y-Richtung
            z = [0,0,1] # z zeigt in globale z-Richtung
        elif image2["axis"] == "-z": # gelb oben
            x = [1,0,0] # x zeigt in globale x-Richtung
            y = [0,-1,0] # y zeigt in globale -y-Richtung
            z = [0,0,-1] # z zeigt in globale -z-Richtung
        elif image2["axis"] == "x": # rot oben
            x = [0,0,1] # x zeigt in globale z-Richtung
            y = [0,-1,0] # y zeigt in globale -y-Richtung
            z = [1,0,0] # z zeigt in globale x-Richtung
        elif image2["axis"] == "-x": # orange oben
            x = [0,0,-1] # x zeigt in globale -z-Richtung
            y = [0,-1,0] # y zeigt in globale -y-Richtung
            z = [-1,0,0] # z zeigt in globale -y-Richtung

    elif image1["axis"] == "-y": # grün links
        if image2["axis"] == "x": # rot oben
            x = [0,0,1] # x zeigt in globale z-Richtung
            y = [0,1,0] # y zeigt in globale y-Richtung
            z = [-1,0,0] # z zeigt in globale -x-Richtung
        elif image2["axis"] == "-x": # orange oben
            x = [0,0,-1] # x zeigt in globale -z-Richtung
            y = [0,1,0] # y zeigt in globale y-Richtung
            z = [1,0,0] # z zeigt in globale x-Richtung
        elif image2["axis"] == "z": # weiß oben
            x = [1,0,0] # x zeigt in globale x-Richtung
            y = [0,1,0] # y zeigt in globale y-Richtung
            z = [0,0,1] # z zeigt in globale z-Richtung
        elif image2["axis"] == "-z": # grün oben
            x = [-1,0,0] # x zeigt in globale -x-Richtung
            y = [0,1,0] # y zeigt in globale y-Richtung
            z = [0,0,-1] # z zeigt in globale -y-Richtung
    
    return (x,y,z)

In [74]:
image1["axis"] = get_axis_by_facecolor(image1)
image2["axis"] = get_axis_by_facecolor(image2)

ksys = get_cube_orientation(image1, image2, rotation1)
print("x: ", ksys[0])
print("y: ", ksys[1])
print("z: ", ksys[2])

x:  [1, 0, 0]
y:  [0, 0, 1]
z:  [0, -1, 0]


Als erster Anlauf schon ganz gut, allerdings sind die Rotationen hart gecoded, die Funktion gilt also aktuell nur für den Fall, dass die erste Rotation um +90° in x-Richtung war. Das muss noch etwas flexibler werden...

## Version 2
Aktuelle Orientierung des Würfels für variable rotationen ermitteln

In [75]:
FACE_AXIS = {
    "R": "y", "O": "-y",
    "W": "z", "Y": "-z",
    "G": "x", "B": "-x",
}

GLOBAL_KSYS = {
    "x": np.array([1, 0, 0]),
    "y": np.array([0, 1, 0]),
    "z": np.array([0, 0, 1]),
}


def get_rotation_matrix(axis:str, steps:int)->np.ndarray:
    """generiert die Rotationsmatrix aus 90° Schritten um die gegebene Achse"""
    angle = steps * (np.pi / 2)
    c, s = np.cos(angle), np.sin(angle)
    if axis == "x":
        return np.array([[1,0,0],[0,c,-s],[0,s,c]])
    if axis == "y":
        return np.array([[c,0,s],[0,1,0],[-s,0,c]])
    if axis == "z":
        return np.array([[c,-s,0],[s,c,0],[0,0,1]])

def rotate_axis(axis:np.ndarray, Mrot:np.ndarray):
    return Mrot @ axis

def get_cube_ksys(image1:dict, image2:dict):
    cube_ksys = {}
    if image1["axis"][0] == "-":
        cube_ksys[image1["axis"][1]] = -image1["orientation"]
        val1 = -image1["orientation"]
    else:
        cube_ksys[image1["axis"]] = +image1["orientation"] 
        val1 = +image1["orientation"]   
    
    if image2["axis"][0] == "-":
        cube_ksys[image2["axis"][1]] = -image2["orientation"]
        val2 = -image2["orientation"]
    else:
        cube_ksys[image2["axis"]] = +image2["orientation"]
        val2 = -image2["orientation"]

    # letzte Achse über Kreuzprodukt berechnen
    val3 = np.cross(val1,val2)

    if cube_ksys.get("x") is None:
        cube_ksys["x"] = val3
    elif cube_ksys.get("y") is None:
        cube_ksys["y"] = val3
    elif cube_ksys.get("z") is None:
        cube_ksys["z"] = val3

    return cube_ksys
    # jetzt noch achse 3 berechnen


### Für Szenario 1 ausführen

In [76]:
# 1. Bild erkannte Achse mappen
image1["axis"] = FACE_AXIS.get(image1["parentcolor"])   # Achse holen (x, y oder z)
image1["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image1["axis"], ": ", image1["orientation"].astype(int))

print(rotation1["steps"]*90, "° Rotation um ", rotation1["axis"])

# 1. Rotation anwenden
Mrot = get_rotation_matrix(rotation1["axis"], rotation1["steps"])
image1["orientation"] = rotate_axis(image1["orientation"], Mrot)

print(image1["axis"], ": ", image1["orientation"].astype(int))

# 2. Bild auswerten
image2["axis"] = FACE_AXIS.get(image2["parentcolor"])   # Achse holen (x, y oder z)
image2["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image2["axis"], ": ", image2["orientation"].astype(int))

print(rotation2["steps"]*90, "° Rotation um ", rotation2["axis"])

# 2. Rotation anwenden
Mrot = get_rotation_matrix(rotation2["axis"], rotation2["steps"])
image1["orientation"] = rotate_axis(image1["orientation"], Mrot)
image2["orientation"] = rotate_axis(image2["orientation"], Mrot)

print(image1["axis"], ": ", image1["orientation"].astype(int))
print(image2["axis"], ": ", image2["orientation"].astype(int))

cube_ksys = get_cube_ksys(image1, image2)

print("\nCalculated Cube ksys rotation matrix")
for key in cube_ksys:
    print(key, ": ", cube_ksys[key].astype(int))

z :  [0 0 1]
90 ° Rotation um  x
z :  [ 0 -1  0]
-x :  [0 0 1]
180 ° Rotation um  y
z :  [ 0 -1  0]
-x :  [ 0  0 -1]

Calculated Cube ksys rotation matrix
z :  [ 0 -1  0]
x :  [0 0 1]
y :  [-1  0  0]


### Für Szenario 2 Ausführen

In [77]:
# 1. Bild erkannte Achse mappen
image1_2["axis"] = FACE_AXIS.get(image1_2["parentcolor"])   # Achse holen (x, y oder z)
image1_2["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image1_2["axis"], ": ", image1_2["orientation"].astype(int))

print(rotation1_2["steps"]*90, "° Rotation um ", rotation1_2["axis"])

# 1. Rotation anwenden
Mrot = get_rotation_matrix(rotation1_2["axis"], rotation1_2["steps"])
image1_2["orientation"] = rotate_axis(image1_2["orientation"], Mrot)

print(image1_2["axis"], ": ", image1_2["orientation"].astype(int))

# 2. Bild auswerten
image2_2["axis"] = FACE_AXIS.get(image2_2["parentcolor"])   # Achse holen (x, y oder z)
image2_2["orientation"] = GLOBAL_KSYS.get("z")            # Aktuelle Ausrichtung beschreiben (wenn erkannt, dann immer z)
print(image2_2["axis"], ": ", image2_2["orientation"].astype(int))

print(rotation2_2["steps"]*90, "° Rotation um ", rotation2_2["axis"])

# 2. Rotation anwenden
Mrot = get_rotation_matrix(rotation2_2["axis"], rotation2_2["steps"])
image1_2["orientation"] = rotate_axis(image1_2["orientation"], Mrot)
image2_2["orientation"] = rotate_axis(image2_2["orientation"], Mrot)

print(image1_2["axis"], ": ", image1_2["orientation"].astype(int))
print(image2_2["axis"], ": ", image2_2["orientation"].astype(int))


cube_ksys = get_cube_ksys(image1_2, image2_2)

print("\nCalculated Cube ksys rotation matrix")
for key in cube_ksys:
    print(key, ": ", cube_ksys[key].astype(int))

-y :  [0 0 1]
90 ° Rotation um  x
-y :  [ 0 -1  0]
x :  [0 0 1]
180 ° Rotation um  y
-y :  [ 0 -1  0]
x :  [ 0  0 -1]

Calculated Cube ksys rotation matrix
y :  [0 1 0]
x :  [ 0  0 -1]
z :  [1 0 0]
